## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [71]:
import pandas as pd
import numpy as np
from datetime import datetime

In [81]:
rate = pd.read_csv('ratings.csv').drop(['movieId'], axis = 1)
sort = rate.pivot_table(index = 'userId', aggfunc = 'count', fill_value = 0)
sort = sort[sort['rating'] > 100].drop(['timestamp'], axis = 1)
table = rate.pivot_table(index = ['userId'], values = 'timestamp', aggfunc = {min, max})
table['lifetime'] = table['max'] - table['min']
lifetime = sort.join(table, how = 'inner')
print('Среднее время жизни пользователя', round((lifetime['lifetime'].mean())/60/60/24), 'дней')


Среднее время жизни пользователя 518 дней


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [342]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [343]:
revenue = client_base.merge(air, how = 'outer').merge(rzd, how = 'outer').merge(auto, how = 'outer').fillna(0)
revenue_noadr = revenue.drop('address', axis = 1)
revenue.head(10)

,client_id,address,air_revenue,rzd_revenue,auto_revenue
0,111,Комсомольская 4,0.0,1093.0,0.0
1,112,Энтузиастов 8а,0.0,2810.0,0.0
2,113,Левобережная 1а,0.0,10283.0,57483.0
3,114,Мира 14,0.0,5774.0,83.0
4,115,ЗЖБИиДК 1,81.0,981.0,912.0
5,116,Строителей 18,4.0,0.0,4834.0
6,117,Панфиловская 33,13.0,0.0,98.0
7,118,Мастеркова 4,173.0,0.0,0.0


In [344]:
revenue_noadr.head(10)

,client_id,air_revenue,rzd_revenue,auto_revenue
0,111,0.0,1093.0,0.0
1,112,0.0,2810.0,0.0
2,113,0.0,10283.0,57483.0
3,114,0.0,5774.0,83.0
4,115,81.0,981.0,912.0
5,116,4.0,0.0,4834.0
6,117,13.0,0.0,98.0
7,118,173.0,0.0,0.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 
1. Как бы вы добавили эти сведения в таблицу визитов и покупок? 
    - Для составления ответа можно использовать вопросы:
        - У каждого пользователя известен набор координат.
        - А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
        - Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. 
2. Как это отразится на вопросе расчетах пункта 1?
    - Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Сделать базу по ключевым регионам с диапазонами координат, которые попадают в нужную геозону (город, район и т.п.)
Координаты пользователя усреднять и подставлять в таблицу одну геозону
Для каждой геозоны нужно расчитать какую-то базовую поправку. Т.е. если вчера визит из Владивостока, а сегодня из Москвы, усреднять это нельзя. Предположим, более 50 км разницы в расчетных координатах - это это уже разные зоны и разные визиты (как вариант). Степень точности усложняет расчеты, но чаще всего от покупателя не требуется суперточного позиционирования, просто понимание из какого он города, или в каком районе он чаще бывает. На основе этих данных можно делать ему таргетированные предложения
Дополнительно можно расчитать временную разницу между отправкой координат, возможно можно сделать вывод, на каком транспорте он передвигается. Частота появления в какой-то точке - места частого пребываения. В какое время чаще всего пользуется приложением/сайтом, в каких районах ему наиболее часто нужно использовать сайт/приложение и т.п.
